# [Immigration to Canada](https://github.com/XutongJiang/ImmigrationToCanada)

### Group members
Section 003: *Xutong Jiang*, *Danny Liu*.  
Section 004: *Jasper Zhao*, *Yan Zeng*.

### Background
This is the final project for ECON 323 at UBC, Vancouver.  
The rising cost of living in Canada is becoming a big problem for many Canadians, especially for new settlers. As an immigrant, choosing the best place to reside in Canada can be a challenging task influenced by a range of factors. Our project aims to provide valuable insights to assist new immigrants in making wise decisions regarding housing rental, income by profession, and living expenses. We achieve this by utilizing data from government sources and presenting it in a visually appealing manner. By offering data-driven insights, our project can guide immigrants as they take their first steps toward settling in Canada.

### Dataset

We gathered 6 datasets from Statistics Canada. These datasets revolve around 3 aspects that we thought were important to our target users (people who are planning their move to Canada). These aspects are rent price, income by occupation and household spending across Canada.  

The datasets are listed below:  

1. https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3410013301 (average rents for areas with a population of 10,000 and over)
2. https://www150.statcan.gc.ca/n1/daily-quotidien/220323/t002a-eng.htm (Median after-tax income, Canada and provinces, 2020)
3. https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=9810045201 (Employment income statistics by occupation minor group, work activity during the reference year, age and gender: Canada, provinces and territories and census metropolitan areas with parts)
4. https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3610022501&pickMembers%5B0%5D=1.11&pickMembers%5B1%5D=2.1&cubeTimeFrame.startYear=2020&cubeTimeFrame.endYear=2020&referencePeriods=20200101%2C20200101 (Detailed household final consumption expenditure, provincial and territorial, annual (x 1,000,000))
5. https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1410032501 (Job vacancies, payroll employees, job vacancy rate, and average offered hourly wage by provinces and territories, quarterly, unadjusted for seasonality)
6. https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1410035601&pickMembers%5B0%5D=1.2&cubeTimeFrame.startMonth=07&cubeTimeFrame.startYear=2021&cubeTimeFrame.endMonth=07&cubeTimeFrame.endYear=2022&referencePeriods=20210701%2C20220701 (Job vacancies and average offered hourly wage by occupation)


In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df1 = pd.read_csv("data/1-Rent.csv")
df2 = pd.read_csv("data/2-Income.csv")
df3 = pd.read_csv("data/3-IncomeByOccupation.csv")
df4 = pd.read_csv("data/4-Spending.csv")
df5 = pd.read_csv("data/5-Vacancy&Wage.csv")
df6 = pd.read_csv("data/6-Occupation&Vacancy&Wage.csv")
province_code = {
  "Newfoundland and Labrador": "NL",
  "Alberta": "AB",
  "British Columbia": "BC",
  "Manitoba": "MB",
  "New Brunswick": "NB",
  "Nova Scotia": "NS",
  "Ontario": "ON",
  "Prince Edward Island": "PE",
  "Quebec": "QC",
  "Saskatchewan": "SK",
}

### Preprocessing
Before implementing the Analysis, it's important to clean up and process the original dataset. We have the following pipeline to preprocess data:
1. Grouping data by provinces to maintain consistancy across datasets. Since in dataset 1, the geographical locations are cities while in all other datasets, geographical locations are provinces, we need to first extract each row’s corresponding province and group the data by their provinces. Finally, we are going to derive attribute Rent by taking the mean of rents of each specific unit type across the province.
2. For datasets 2, 4 and 5, we are joining them together on the Province attribute. The number of rows won’t change since in the original datasets, each province is corresponding to one row and one quantitative attribute value.
3. We will also join dataset 3 and 6, by the unique identifier Province + Occupation. For occupations, we keep only the top levels in this table, as the corresponding visualization would only need that degree of information.
4. Lastly, we transform each province name to province code instead, as some names are quite long and later may cause label overlapping on the visualization.

In [23]:
# Step 1
df1['Province'] = df1['Province'].apply(lambda x: x.split(', ')[1])
df1 = df1.groupby(['Province','Unit'])['Rent'].mean().reset_index()

# Step 2
df245 = pd.merge(df2, df4, on='Province').merge(df5, on='Province')

# Step 3
df36 = pd.merge(df3, df6, on=['Province','Occupation'])

# Step 4
def translate_province(df):
    df['Province'] = df['Province'].apply(lambda x: province_code.get(x))
translate_province(df1)
translate_province(df245)
translate_province(df36)

In [24]:
df1

,Province,Unit,Rent
0,AB,Bachelor units,852.400000
1,AB,One bedroom units,1005.470588
2,AB,Three bedroom units,1301.125000
3,AB,Two bedroom units,1185.210526
4,BC,Bachelor units,908.700000
5,BC,One bedroom units,1054.269231
6,BC,Three bedroom units,1538.739130
7,BC,Two bedroom units,1299.961538
8,MB,Bachelor units,670.400000
9,MB,One bedroom units,871.200000


In [25]:
df245

,Province,Income,Spending,Vacancy,Rate,Wage
0,NL,59300,14930.4,"8,185",3.7,22.30
1,PE,59400,4207.1,"4,090",5.4,19.50
2,NS,57500,26161.7,"22,960",5.2,20.80
3,NB,56900,21062.6,"16,430",4.7,20.90
4,QC,59700,220364.2,"246,230",6.1,23.70
5,ON,70100,399266.3,"372,075",5.4,24.65
6,MB,63000,34290.8,"32,290",5.1,21.60
7,SK,67700,31716.4,"25,445",5.1,22.45
8,AB,77700,126489.2,"103,380",5.0,24.80
9,BC,67500,149984.6,"156,645",6.2,25.45


In [26]:
df36

,Province,Occupation,Income,Vacancy,Wage
0,NL,Legislative and senior management,97000,275,33.30
1,NL,"Business, finance and administration",47200,595,23.25
2,NL,Natural and applied sciences,78500,295,35.00
3,NL,Health,55200,"1,480",27.45
4,NL,"Education, Law and Public Services",44800,710,20.65
...,...,...,...,...,...
95,BC,"Art, Culture, Recreation and Sport",25200,"2,815",30.55
96,BC,Sales and service,23200,"44,485",18.50
97,BC,"Trades, Transport and Equipment operators",50400,"26,605",27.95
98,BC,"Natural resources, Agriculture and Production",26800,"3,350",23.85
